# Startup for France
This code is to be included at the beginning of `calib_FRA` and `simu_FRA`.

In [1]:
t0 = time() ## start chrono
dir_module = joinpath(pwd(),"src")
dir_graph = joinpath(pwd(),"../../graphs")
    
########### main module
using NBInclude
@nbinclude(joinpath(dir_module,"AgeDisc.ipynb"))
using Main.AgeDisc

########### other packages
using QuadGK, DataFrames, PyPlot

# Moments

In [2]:
data_fra = DataFrame([
(20, 0.638),
(21, 0.634),
(22, 0.659),
(23, 0.683),
(24, 0.721),
(25, 0.664),
(26, 0.656),
(27, 0.676),
(28, 0.678),
(29, 0.676),
(30, 0.682),
(31, 0.673),
(32, 0.677),
(33, 0.673),
(34, 0.692),
(35, 0.705)], [:age, :rate])
data_fra.age *=12


moments_target, Age_vec = let
    Age_bds = (24,30) .*12 ## window to match the moments
    myselec = Age_bds[1] .<= data_fra.age .<= Age_bds[2]
    (data_fra.rate[myselec], data_fra.age[myselec])
end


function moments_france(ureal; 
            atol=1e-8, a_range=Age_vec)
    Y_vec = [quadgk(x->1-sum(ureal(x)[:]) ,a, a+12, atol=atol)[1]/12 for a in a_range]
    return Y_vec
end


function UIpolicy(jobspell) ## from DARES in 1999, caveat: we only account for continuous employment spell
    if jobspell < 4 ## no eligible below 4 months
        UImax = 0.
    elseif jobspell < 6
        UImax = 4
    elseif jobspell < 8
        UImax = 7
    elseif jobspell < 14
        UImax = 15
    else
        UImax = 30
    end
    return(UImax)
end

lambda_function(q) = 1/quadgk(x-> UIpolicy(x) * q * exp(-q*x), 0, Inf)[1] 
#lambda_function(q0,qa,tmax) = 1/quadgk(x-> UIpolicy(x) * (q0+qa*x) * exp(-(q0+qa*x)*x), 0, tmax)[1] 

lambda_function (generic function with 1 method)

# Parameters

In [3]:
C0 = let
    ## incomes
    b1_obs = 539 # RMI+housing benefits, from BD2011
    b0_obs = 206 # housing benefits without RMI, from BD2011
    wbar_obs = 1.66 * b1_obs # average wage, 66% increase in relative gains from working for HS dropouts, from BD2011    

    Country1(
        A = 25. * 12, Amin = 18. *12, Amax = 36. *12, Aret = 65. * 12,
        r = 0.0048, # 5 % annual interest rate, time unit = month
        α = NaN, m = NaN, q0 = NaN, qa=NaN, λ = NaN, ξ=0.,
        uinit = [1.,0.], ## start with 0% employment rate at Amin
        b1 = b1_obs / wbar_obs, b0 = b0_obs / wbar_obs, 
        bUI = 0.68,   # 68% replacement rate from DARES,
        B1=NaN, B2=NaN, B1dif=NaN
    )    
end

function update_Country1(;α=C0.α, m=C0.m, q0=C0.q0, qa=C0.qa, ξ=C0.ξ, Aret=C0.Aret)
    (B1, B1dif, B2) = Baux(α=α, m=m) 
    Country1(A=C0.A, Amin=C0.Amin, Amax=C0.Amax, Aret=Aret,
                    r=C0.r, q0=q0, qa=qa, m=m, λ=lambda_function(q0), α=α, ξ=ξ,
                    bUI=C0.bUI, b0=C0.b0, b1=C0.b1,
                    uinit=C0.uinit,
                    B1=B1, B1dif=B1dif, B2=B2)
end

update_Country1 (generic function with 1 method)

In [4]:
println("C0 is $C0")
t1 = time()
nsec = (t1-t0)
println("Startup run in $nsec secs.")

C0 is Country1(300.0, 216.0, 432.0, 780.0, 0.0048, NaN, NaN, NaN, NaN, NaN, 0.0, [1.0, 0.0], 0.68, 0.6024096385542168, 0.23023448152535933, NaN, NaN, NaN)
Startup run in 89.07390093803406 secs.
